### Install the required dependencies
i am using Mistral AI and Sqlite

In [ ]:
# %pip install -U langchain-community langgraph langchain-mistralai tavily-python langgraph-checkpoint-sqlite

Setup [Langsmith](https://www.langchain.com/langsmith) and [Tavily API Key](https://tavily.com/)

In [ ]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key for LangSmith: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter API key for Tavily: ")

Tavily search engine as tool.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is weather in Delhi")
# print(search_results) - give results in JSON
tools = [search] 
# Assigning search to tools allows you to pass this list to an 
# LLM-powered agent so it can decide which tool to use.

json_to_string_results = "\n".join([
    f"{idx+1}.{result['title']}-{result['content']}" 
    for idx,result in enumerate(search_results)
    ])
print(json_to_string_results)

1.Weather in Delhi, India-{'location': {'name': 'Delhi', 'region': 'Delhi', 'country': 'India', 'lat': 28.6667, 'lon': 77.2167, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1743221809, 'localtime': '2025-03-29 09:46'}, 'current': {'last_updated_epoch': 1743221700, 'last_updated': '2025-03-29 09:45', 'temp_c': 24.9, 'temp_f': 76.7, 'is_day': 1, 'condition': {'text': 'Partly Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 14.1, 'wind_kph': 22.7, 'wind_degree': 306, 'wind_dir': 'NW', 'pressure_mb': 1011.0, 'pressure_in': 29.87, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 7, 'cloud': 33, 'feelslike_c': 23.7, 'feelslike_f': 74.6, 'windchill_c': 24.9, 'windchill_f': 76.7, 'heatindex_c': 23.7, 'heatindex_f': 74.6, 'dewpoint_c': -13.1, 'dewpoint_f': 8.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 3.8, 'gust_mph': 16.2, 'gust_kph': 26.1}}
2.Delhi Weather Today, Delhi Temperature and Air Quality (2025-03-29)-Delhi Weather Today (Saturday, Mar 29, 202

### Using LLM

Install the language model (can change based on your preference)

In [ ]:
# pip install -qU "langchain[mistralai]"

In [ ]:
from langchain.chat_models import init_chat_model
# checks if key is avaliable  and if not, ask user to enter one
if not os.environ.get("MISTRAL_API_KEY"):
    os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral AI: ")

model = init_chat_model("mistral-large-latest", model_provider="mistralai")
# 

You can call the language model by passing in a list of messages. By default, the response is a content string.

In [10]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

"Hello! How can I assist you today? Let's have a friendly conversation. 😊 How are you doing?"

In [11]:
# to enable this model to do tool calling we use .bind_tools
# to give the language model knowledge of these tools
model_with_tools = model.bind_tools(tools)

In [12]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")
# there was no tool called for this input to process so it returned empty

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [13]:
# Now, let's try calling it with some input that would expect a tool to be called.
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Delhi?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

# Many LLMs (including Mistral and OpenAI's models) 
# will decide whether to call a tool only if they think it's necessary.

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Delhi'}, 'id': 'H6yI33aHA', 'type': 'tool_call'}]


Using ReAct(Reasoning + Acting) agent

In [14]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [15]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})
response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='79184db7-fae0-445f-bb3e-3c0e82ffc7f0'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 109, 'total_tokens': 119, 'completion_tokens': 10}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-e81d8376-f18f-441c-b3f3-332d651dd6a9-0', usage_metadata={'input_tokens': 109, 'output_tokens': 10, 'total_tokens': 119})]

In [16]:
response = agent_executor.invoke({"messages": [HumanMessage(content="What is the weather in Delhi?")]})
response["messages"]

[HumanMessage(content='What is the weather in Delhi?', additional_kwargs={}, response_metadata={}, id='459683ea-e6fd-417d-b6d7-1245bda8ea8e'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'PszTYxLw4', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in Delhi"}'}, 'index': 0}]}, response_metadata={'token_usage': {'prompt_tokens': 114, 'total_tokens': 145, 'completion_tokens': 31}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-ac33f8f9-e045-410e-8fd2-716df1c23ee8-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Delhi'}, 'id': 'PszTYxLw4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 114, 'output_tokens': 31, 'total_tokens': 145}),
 ToolMessage(content='[{"title": "Weather in Delhi, India", "url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Delhi\', \'region\': \'Delhi\', \'country\': \'India\', \'la

invkoing tool calls

In [17]:
# If the agent executes multiple steps, this may take a while. 
# To show intermediate progress, we can stream back messages as they occur.

for step in agent_executor.stream(
    {"messages": [HumanMessage(content="What is the weather in Delhi?")]},
    stream_mode="values", 
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the weather in Delhi?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (aFKxznpC5)
 Call ID: aFKxznpC5
  Args:
    query: weather in Delhi
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Weather in Delhi, India", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Delhi', 'region': 'Delhi', 'country': 'India', 'lat': 28.6667, 'lon': 77.2167, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1743227871, 'localtime': '2025-03-29 11:27'}, 'current': {'last_updated_epoch': 1743227100, 'last_updated': '2025-03-29 11:15', 'temp_c': 29.5, 'temp_f': 85.0, 'is_day': 1, 'condition': {'text': 'Partly Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 13.2, 'wind_kph': 21.2, 'wind_degree': 311, 'wind_di

Streaming Messages

In [18]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="What is the weather in Delhi?")]},
    stream_mode="messages", 
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| current| weather| in| Delhi| is| partly| cloud|y| with| a| temperature| of| |2|9|.|5|°|C| (|8|5|.|0|°|F|).| The| wind| is| blowing| at| |2|1|.|2| km|/|h| from| the| north|west|,| and| the| hum|idity| is| at| |4|%.| The| feels|-|like| temperature| is| |2|7|.|3|°|C| (|8|1|.|1|°|F|).| There| is| no| precip|itation| at| the| moment|,| and| the| UV| index| is| |6|.|5| |.|
|
|For| the| upcoming| days| in| March| |2|0|2|5|,| the| expected| temperatures| and| precip|itation| are| as| follows|:|
|-| **|M|arch| |2|9|:**| High| of| |3|2|°|C| (|9|0|°|F|),| Low| of| |1|8|°|C| (|6|4|°|F|),| Prec|ip|itation|:| |0|.|4| mm| (|0|.|0| inches|)|
|-| **|M|arch| |3|0|:**| High| of| |3|2|°|C| (|9|0|°|F|),| Low| of| |1|8|°|C| (|6|4|°|F|),| Prec|ip|itation|:| |1|.|3| mm| (|0|.|1| inches|)|
|-| **|M|arch| |3|1|:**| High| of| |3|3|°|C| (|9|1|°|F|),| Low| of| |1|7|°|C| (|6|3|°|F|),| Prec|ip|itation|:| |1|.|6| mm| (|0|.|1| inches|)| |.|

Adding in Memory

In [19]:
# this agent is stateless.
# This means it does not remember previous interactions.
# To give it memory we need to pass in a checkpointer.

from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

Setting checkpoint

In [25]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

In [27]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
    ):
        print(chunk)
        print("----")

{'agent': {'messages': [AIMessage(content="Hey Bob! It's good to see you again.", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 209, 'total_tokens': 222, 'completion_tokens': 13}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-1de7a6e9-2318-46e1-b252-708b99ab4201-0', usage_metadata={'input_tokens': 209, 'output_tokens': 13, 'total_tokens': 222})]}}
----


If you want to start a new conversation, all you have to do is change the ```thread_id``` used

In [28]:
config = {"configurable": {"thread_id": "xyz123"}} # New Thread ID
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What is my name?")]}, config
    ):
       print(chunk)
       print("----")

{'agent': {'messages': [AIMessage(content="I don't have access to personal information about you, including your name. If you have any other questions or need assistance with something else, feel free to ask!", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 112, 'total_tokens': 147, 'completion_tokens': 35}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-e32c2ece-d480-4634-8a23-a2486908b569-0', usage_metadata={'input_tokens': 112, 'output_tokens': 35, 'total_tokens': 147})]}}
----
